In [2]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import xarray as xr 
import scipy.stats as stat
import collections
import sys
import os
import glob
import re
import importlib
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.magnum.database as ut
import flopter.magnum.utils as mgut
import flopter.core.fitters as fts
import flopter.core.constants as c
import analysis.scripts.magnum.magnum_advanced_analysis as maa
import flopter.magnum.magoptoffline as mg

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:215: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:215: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:215: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


In [24]:
# adc_index = 276
# shot_number = 284
# shot_number = 293
shot_number = 317

In [25]:
# Create analysed dataset metadata 

path_to_datasets = '/home/jleland/data/external/magnum/'
# path_to_datasets = '/home/jleland/data/externy/magnum/'
# path_to_analysed_datasets = 'analysed_2'
# path_to_analysed_datasets = 'analysed_3'
# path_to_analysed_datasets = 'phobos_test'
# path_to_analysed_datasets = 'test'
# path_to_analysed_datasets = 'analysed_3_downsampled'
# path_to_analysed_datasets = 'analysed_4'
path_to_analysed_datasets = 'analysed_4_downsampled'
# path_to_analysed_datasets = 'analysed_5_downsampled'
os.chdir(path_to_datasets)

In [5]:
magnum_probes = lp.MagnumProbes()

## Load adc file metadata

In [6]:
os.chdir('/home/jleland/data/external/magnum/')
# os.chdir('/home/jleland/data/externy/magnum/')
meta_data_ds = xr.open_dataset('all_meta_data.nc')
# print(meta_data_ds)

# Plot whole time series to see sweep partitioning

In [26]:
ds = mgut.get_dataset_from_indices([shot_number], anglescan_fl=False, average_direction_fl=False, 
                                   path_to_analysed_datasets=path_to_analysed_datasets,
                                   preprocess='autosel').isel(shot_number=0)

In [27]:
ds

<xarray.Dataset>
Dimensions:                 (direction: 2, probe: 2, sweep: 758, time: 50, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'S' 'L'
  * direction               (direction) object 'up' 'down'
    shot_number             int32 325
    ts_number               float64 ...
    ts_timestamp            float64 ...
    ts_time                 datetime64[ns] ...
    adc_index               float64 ...
    adc_time                datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    float64 -0.0
Dimensions without coordinates: sweep, time
Data variables:
    voltage                 (probe, direction, sweep, time) float64 3.241 ... -94.69
    current                 (probe, direction, sweep, time) float64 0.0467 ... -0.001193
    shot_time               (probe, direction, sweep, time) float64 1.001 ... 8.576
    start_time              (probe, direction, sweep) float64 1.001 ... 8.576
    adc_filename            object ...
    ts_density              (ts_radial_pos) float64 ...
    ts_temperature          (ts_radial_pos) float64 ...
    ts_d_density            (ts_radial_pos) float64 ...
    ts_d_temperature        (ts_radial_pos) float64 ...
    adc_folder              object ...
    adc_calibration_index   object ...
    adc_4_probe             object ...
    adc_5_probe             object ...
    ts_temp_max             float64 ...
    ts_dens_max             float64 ...
    shot_end_time           datetime64[ns] ...
    ts_fl                   bool ...
    sweep_time              datetime64[ns] ...
    shot_source_current     float64 ...
    shot_b_field            float64 ...
    shot_tilt               float64 -0.0355
    shot_target_pos         float64 ...
    shot_target_rot         float64 ...
    shot_target_pumpspeed   int32 ...
    shot_source_pumpspeed   int32 ...
    shot_heating_pumpspeed  int32 ...
    shot_source_pos         float64 ...
    shot_deuterium_gf       float64 ...
    shot_hydrogen_gf        float64 ...
    shot_helium_gf          float64 ...
    shot_hydrogen_puffed    float64 ...
    shot_helium_puffed      float64 ...
    adc_timestamp           float64 ...
    adc_4_coax              float64 ...
    adc_4_shunt_resistance  float64 ...
    adc_5_coax              float64 ...
    adc_5_shunt_resistance  float64 ...
    adc_calibration_fl      float64 ...
    shot_fl                 float64 ...
    acquisition_length      float64 ...
    adc_end_time            datetime64[ns] ...
    adc_end_timestamp       float64 ...
    adc_freqs               float64 ...

In [28]:
ds.mean('direction').max('time')['current'].plot.line(hue='probe')

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:215: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


In [29]:
fig, ax = plt.subplots(2, sharex=True)

probe_ds = ds.sel(probe='L').mean('direction')
for i, sweep in enumerate(probe_ds.sweep.values[:500]):
    sweep_ds = probe_ds.sel(sweep=sweep)
    time_offset = (i*sweep_ds.time.size)
    ax[0].plot(sweep_ds['time'] + time_offset, sweep_ds['current'])
    ax[1].plot(sweep_ds['time'] + time_offset, sweep_ds['voltage'])

# Look at Raw Data for a problem sweep

In [30]:
shot_info = maa.get_shot_info_for_analysis(shot_number)
shot_info

('2019-06-05_Leland/',
 '2019-06-05 13h 59m 39s TT_06699014924537268256.adc',
 'a317_310_6699014924537268224',
 array(0.98926088),
 array(4.55653029e+20),
 ('S', 'L'),
 array(1.),
 100,
 (10.04, 10.04),
 (1, 2350))

In [31]:
folder, adc_file, output_tag, ts_temp, ts_dens, probe_designations, shunt_resistance, downsampling_factor, cabling_resistance, sweep_range = shot_info

In [32]:
mg.Magoptoffline._FOLDER_STRUCTURE = 'data/external/magnum/'
print('"{}" \t\t "{}"'.format(folder, adc_file))

# Create magopter object
print('Creating magopter object')
magopter = mg.Magoptoffline(folder, adc_file, shunt_resistor=shunt_resistance, cabling_resistance=cabling_resistance)
magopter._VOLTAGE_CHANNEL = 3
magopter._PROBE_CHANNEL_3 = 4
magopter._PROBE_CHANNEL_4 = 5
# magopter.prepare(down_sampling_rate=downsampling_factor, roi_b_plasma=True, filter_arcs_fl=False, crit_freq=45000, crit_ampl=None, homogenise_fl=False)
magopter.prepare(down_sampling_rate=downsampling_factor, roi_b_plasma=True, filter_arcs_fl=False, crit_freq=None, crit_ampl=None, homogenise_fl=False)

"2019-06-05_Leland/" 		 "2019-06-05 13h 59m 39s TT_06699014924537268256.adc"
Creating magopter object
Running an offline magopter object, operating in offline mode.


In [37]:
fig, ax = plt.subplots()

print(f'raw_time: {len(magopter.raw_time)}, raw_voltage: {len(magopter.raw_voltage)}')

# make a slice of interest
slc_oi = slice(0, int(0.05 * len(magopter.raw_time)))
ax.plot(magopter.raw_time[slc_oi], magopter.raw_voltage[slc_oi])
ax.plot(magopter.raw_time[slc_oi], magopter.voltage[0][slc_oi])
ax.plot(magopter.raw_time[slc_oi], magopter.voltage[1][slc_oi])

raw_time: 225000, raw_voltage: 225000


In [47]:
fig, ax = plt.subplots()

print(f'raw_time: {len(magopter.raw_time)}, raw_voltage: {len(magopter.raw_voltage)}')

# make a slice of interest
slc_oi = slice(0, int(0.05 * len(magopter.raw_time)))
ax.plot(magopter.raw_time[slc_oi], magopter.raw_current[0][slc_oi])
ax.plot(magopter.raw_time[slc_oi], magopter.current[0][slc_oi])
# ax.plot(magopter.raw_time[slc_oi], magopter.current[1][slc_oi])

raw_time: 225000, raw_voltage: 225000


In [34]:
magopter.homogenise()

[ 1.0023    1.0073    1.0124   ... 23.487201 23.492201 23.497201]
47.411690581659165 0.009999955990119306 -45.68267073236613 0.0023000001022221994


In [43]:
fig, ax = plt.subplots(2)
iv_data = magopter.iv_arrs[1][2]
ax[0].plot(iv_data['V'])
ax[1].plot(iv_data['I'])

In [45]:
fit, ax = plt.subplots(2, sharex=True)

for iv_data in magopter.iv_arrs[0][:100]:
    ax[0].plot(iv_data['t'], iv_data['I'])
    ax[0].axvline(iv_data['t'][0], linewidth=0.7, color='gray', linestyle=':')
    
    ax[1].plot(iv_data['t'], iv_data['V'])
    ax[1].axvline(iv_data['t'][0], linewidth=0.7, color='gray', linestyle=':')

In [21]:
ds_full = magopter.to_xarray(('S', 'L'))

In [23]:
ds_full.mean(['direction', 'sweep']).sel(probe='S').set_coords('voltage')['current'].plot.line(x='voltage')